In [1]:
import os
import numpy as np
import torch
import warnings

warnings.filterwarnings("ignore")
if os.getcwd()[-8:]=='examples':
    os.chdir('..')

os.environ["CUDA_VISIBLE_DEVICES"] = "1" # Possible GPUS

In [2]:
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import importlib

from opacus import PrivacyEngine
from opacus.validators import ModuleValidator

import src
import src.trainer as tr

### Settings

In [3]:
#### FOR DP
MAX_GRAD_NORM = 0.1
EPSILON = 3.0
DELTA = 1e-5

#### FOR TRAINING
LR = 2
EPOCHS = 30
BATCH_SIZE = 2048
MAX_PHYSICAL_BATCH_SIZE = 1024
RHO = 0.03

#### FOR SAVE
PATH = "./saved/"
NAME = "CIFAR10_DPNAS_EPS3"
SAVE_PATH = PATH + NAME

### FOR MODELING
MODEL_NAME = "DPNASNet_CIFAR"
DATA = "CIFAR10"
NORMALIZE = {'mean':[0.4914, 0.4822, 0.4465],
             'std':[0.2023, 0.1994, 0.2010]}
N_CLASSES = 10

### Data loader

In [4]:
data = src.Datasets(data_name=DATA, train_transform = transforms.ToTensor())
train_loader, test_loader = data.get_loader(batch_size=BATCH_SIZE, drop_last_train=False, num_workers=16)

Files already downloaded and verified
Files already downloaded and verified
Data Loaded!
Train Data Length : 50000
Test Data Length : 10000


### Model & Optimizer

In [5]:
#### Load model
model = src.utils.load_model(model_name=MODEL_NAME, n_classes=N_CLASSES).cuda() # Load model
model = ModuleValidator.fix(model)
ModuleValidator.validate(model, strict=False)

pytorch_total_params = sum(p.numel() for p in model.parameters())

print("model params: {:.4f}M".format(pytorch_total_params/1000000))

#### Define optimizer
optimizer = torch.optim.SGD(model.parameters(),lr=LR, momentum=0.9)

[(5, 0, 1), (1, 0, 2), (5, 1, 2), (6, 0, 3), (1, 1, 3), (0, 2, 3), (5, 0, 4), (0, 1, 4), (0, 2, 4), (3, 3, 4), (5, 0, 5), (5, 1, 5), (6, 2, 5), (1, 3, 5), (6, 4, 5)]
DPNASNet_CIFAR is loaded.
model params: 0.5294M


### Load PrivacyEngine from Opacus

In [6]:
privacy_engine = PrivacyEngine()

model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
)

optimizer.target_epsilon = EPSILON
optimizer.target_delta = DELTA

print(f"Using sigma={optimizer.noise_multiplier} and C={MAX_GRAD_NORM}")
# 2.0664

rmodel = src.RobModel(model, n_classes=N_CLASSES, normalize=NORMALIZE).cuda()

Using sigma=1.8408203125 and C=0.1


### Start Training

In [7]:
importlib.reload(tr)

trainer = tr.DpTrainer(NAME,rmodel)
trainer.max_physical_batch_size = MAX_PHYSICAL_BATCH_SIZE
trainer.record_rob(train_loader, test_loader)

In [8]:
trainer.fit(train_loader=train_loader, max_epoch=EPOCHS, start_epoch=0,
            optimizer=optimizer,
            scheduler=None, scheduler_type="Epoch",
            minimizer="DPSAT(rho={})".format(RHO),
            save_path=SAVE_PATH, save_best={"Clean(Val)":"HB"},
            save_type=None, save_overwrite=True, record_type="Epoch")

[CIFAR10_DPNAS_EPS3]
Training Information.
-Epochs: 30
-Optimizer: SGD (
Parameter Group 0
    dampening: 0
    lr: 2
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)
-Scheduler: None
-Save Path: ./saved/CIFAR10_DPNAS_EPS3
-Save Type: None
-Record Type: Epoch
-Device: cuda:0
----------------------------------------------------------
Epoch   CALoss^p   Clean(Tr)   Clean(Val)   lr   Sec/it   
1       2.0424     36.8527     37.7800      2    0.3970   
----------------------------------------------------------
2       1.6803     45.2495     44.8500      2    0.6731   
----------------------------------------------------------
3       1.7365     48.0667     47.9100      2    0.6513   
----------------------------------------------------------
4       1.6374     50.6267     49.9100      2    0.6639   
----------------------------------------------------------
5       1.6274     55.9413     55.9600      2    0.6425   
----------------------------------------------------------
6    

### Evaluation

In [9]:
rmodel.load_dict(PATH+NAME+'/last.pth')
rmodel.eval_accuracy(test_loader)

Model loaded.
Record Info:
OrderedDict([('Epoch', 30), ('CALoss^p', 1.5596732544898986), ('Clean(Tr)', 71.70541862089611), ('Clean(Val)', 69.62), ('lr', 2)])


69.62

In [10]:
rmodel.load_dict(PATH+NAME+'/best.pth')
rmodel.eval_accuracy(test_loader)

Model loaded.
Record Info:
OrderedDict([('Epoch', 28), ('CALoss^p', 1.5732993174482275), ('Clean(Tr)', 71.42340532395781), ('Clean(Val)', 69.66), ('lr', 2)])


69.66